# SPY Monte Carlo — 1-Year GBM
Pulls 5y of SPY daily data, estimates μ and σ, runs Monte Carlo for one year (252 trading days), and produces two visuals: percentile paths and a one-year return histogram.

In [ ]:
# Imports & setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from pathlib import Path

SEED = 42
np.random.seed(SEED)

FIG_DIR = Path('figures')
FIG_DIR.mkdir(exist_ok=True)

plt.rcParams['figure.figsize'] = (10, 4.8)
plt.rcParams['axes.grid'] = True


In [ ]:
# Pull last 5 years of SPY daily data
end = pd.Timestamp.today().normalize()
start = end - pd.Timedelta(days=int(365.25*5)+7)

spy = yf.download('SPY', start=start, end=end, progress=False)
spy = spy[['Adj Close']].rename(columns={'Adj Close':'adj_close'}).dropna()
spy.tail()

In [ ]:
# Daily log-returns and annualized μ, σ
prices = spy['adj_close']
logret = np.log(prices).diff().dropna()

TRADING_DAYS = 252
mu_d = logret.mean()
sigma_d = logret.std(ddof=1)

mu = mu_d * TRADING_DAYS
sigma = sigma_d * np.sqrt(TRADING_DAYS)
S0 = float(prices.iloc[-1])

mu, sigma, S0

In [ ]:
# Monte Carlo GBM
N = 1000
T = TRADING_DAYS
dt = 1.0 / TRADING_DAYS

eps = np.random.normal(size=(T, N))
paths = np.zeros((T+1, N))
paths[0, :] = S0

drift = (mu - 0.5*sigma**2)*dt
diff = sigma*np.sqrt(dt)

for t in range(1, T+1):
    paths[t, :] = paths[t-1, :] * np.exp(drift + diff*eps[t-1, :])

pcts = [5,25,50,75,95]
percs = np.percentile(paths, pcts, axis=1)
mean_path = paths.mean(axis=1)


In [ ]:
# Figure 1: Percentile paths
days = np.arange(T+1)
fig, ax = plt.subplots()
labels = ['p5','p25','p50 (median)','p75','p95']
for i, lbl in enumerate(labels):
    ax.plot(days, percs[i], label=lbl, linewidth=2 if '50' in lbl else 1.8)
ax.plot(days, mean_path, label='mean', linestyle='--', linewidth=1.5)
ax.set_title('SPY — One-Year Monte Carlo Percentile Paths')
ax.set_xlabel('Day'); ax.set_ylabel('Price'); ax.legend(ncol=3, fontsize=9)
fig.tight_layout()

ts = datetime.now().strftime('%Y%m%d_%H%M%S')
fig1_path = FIG_DIR / f'percentile_paths_{ts}.png'
plt.savefig(fig1_path, dpi=200)
fig1_path

In [ ]:
# Figure 2: One-year return histogram
terminal = paths[-1, :]
terminal_ret = terminal / S0 - 1.0

fig, ax = plt.subplots()
ax.hist(terminal_ret, bins=40, alpha=0.85)
for b in np.percentile(terminal_ret, [5,25,50,75,95]):
    ax.axvline(b, linestyle='--', linewidth=1.3)
ax.set_title('SPY — Simulated One-Year Return Distribution')
ax.set_xlabel('Return'); ax.set_ylabel('Frequency')
fig.tight_layout()

fig2_path = FIG_DIR / f'return_histogram_{ts}.png'
plt.savefig(fig2_path, dpi=200)
(fig2_path, np.percentile(terminal_ret, [5,25,50,75,95]))

**Notes**  
- The mean path typically lies above the median under lognormal dynamics (skew).  
- Re-run periodically to refresh μ, σ estimates; bands will change with volatility.  
- Increase `N` for smoother histograms (runtime scales linearly).
